# [1341. Movie Rating](https://leetcode.com/problems/movie-rating/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Movies

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| movie_id      | int     |
| title         | varchar |
+---------------+---------+</pre>
movie_id is the primary key (column with unique values) for this table.
title is the name of the movie.


Table: Users

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| name          | varchar |
+---------------+---------+</pre>
user_id is the primary key (column with unique values) for this table.


Table: MovieRating

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| movie_id      | int     |
| user_id       | int     |
| rating        | int     |
| created_at    | date    |
+---------------+---------+</pre>
(movie_id, user_id) is the primary key (column with unique values) for this table.
This table contains the rating of a movie by a user in their review.
created_at is the user's review date.


Write a solution to:

Find the name of the user who has rated the greatest number of movies. In case of a tie, return the lexicographically smaller user name.
Find the movie name with the highest average rating in February 2020. In case of a tie, return the lexicographically smaller movie name.
The result format is in the following example.



Example 1:

Input:
Movies table:
<pre>+-------------+--------------+
| movie_id    |  title       |
+-------------+--------------+
| 1           | Avengers     |
| 2           | Frozen 2     |
| 3           | Joker        |
+-------------+--------------+</pre>
Users table:
<pre>+-------------+--------------+
| user_id     |  name        |
+-------------+--------------+
| 1           | Daniel       |
| 2           | Monica       |
| 3           | Maria        |
| 4           | James        |
+-------------+--------------+</pre>
MovieRating table:
<pre>+-------------+--------------+--------------+-------------+
| movie_id    | user_id      | rating       | created_at  |
+-------------+--------------+--------------+-------------+
| 1           | 1            | 3            | 2020-01-12  |
| 1           | 2            | 4            | 2020-02-11  |
| 1           | 3            | 2            | 2020-02-12  |
| 1           | 4            | 1            | 2020-01-01  |
| 2           | 1            | 5            | 2020-02-17  |
| 2           | 2            | 2            | 2020-02-01  |
| 2           | 3            | 2            | 2020-03-01  |
| 3           | 1            | 3            | 2020-02-22  |
| 3           | 2            | 4            | 2020-02-25  |
+-------------+--------------+--------------+-------------+</pre>
Output:
<pre>+--------------+
| results      |
+--------------+
| Daniel       |
| Frozen 2     |
+--------------+</pre>
Explanation:
Daniel and Monica have rated 3 movies ("Avengers", "Frozen 2" and "Joker") but Daniel is smaller lexicographically.
Frozen 2 and Joker have a rating average of 3.5 in February but Frozen 2 is smaller lexicographically.

In [1]:
# Pandas schema
import pandas as pd

data = [[1, 'Avengers'], [2, 'Frozen 2'], [3, 'Joker']]
movies = pd.DataFrame(data, columns=['movie_id', 'title']).astype({'movie_id': 'Int64', 'title': 'object'})
data = [[1, 'Daniel'], [2, 'Monica'], [3, 'Maria'], [4, 'James']]
users = pd.DataFrame(data, columns=['user_id', 'name']).astype({'user_id': 'Int64', 'name': 'object'})
data = [[1, 1, 3, '2020-01-12'], [1, 2, 4, '2020-02-11'], [1, 3, 2, '2020-02-12'], [1, 4, 1, '2020-01-01'],
        [2, 1, 5, '2020-02-17'], [2, 2, 2, '2020-02-01'], [2, 3, 2, '2020-03-01'], [3, 1, 3, '2020-02-22'],
        [3, 2, 4, '2020-02-25']]
movie_rating = pd.DataFrame(data, columns=['movie_id', 'user_id', 'rating', 'created_at']).astype(
    {'movie_id': 'Int64', 'user_id': 'Int64', 'rating': 'Int64', 'created_at': 'datetime64[ns]'})

#to spark dataframe
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

movies_df = spark.createDataFrame(movies)
users_df = spark.createDataFrame(users)
movie_rating_df = spark.createDataFrame(movie_rating)

In [3]:
movies_df.show()

+--------+--------+
|movie_id|   title|
+--------+--------+
|       1|Avengers|
|       2|Frozen 2|
|       3|   Joker|
+--------+--------+



In [4]:
users_df.show()

+-------+------+
|user_id|  name|
+-------+------+
|      1|Daniel|
|      2|Monica|
|      3| Maria|
|      4| James|
+-------+------+



In [5]:
movie_rating_df.show()

+--------+-------+------+-------------------+
|movie_id|user_id|rating|         created_at|
+--------+-------+------+-------------------+
|       1|      1|     3|2020-01-12 00:00:00|
|       1|      2|     4|2020-02-11 00:00:00|
|       1|      3|     2|2020-02-12 00:00:00|
|       1|      4|     1|2020-01-01 00:00:00|
|       2|      1|     5|2020-02-17 00:00:00|
|       2|      2|     2|2020-02-01 00:00:00|
|       2|      3|     2|2020-03-01 00:00:00|
|       3|      1|     3|2020-02-22 00:00:00|
|       3|      2|     4|2020-02-25 00:00:00|
+--------+-------+------+-------------------+



In [20]:
from collections import OrderedDict
from pyspark.sql import Window

# Solving in spark dataframe
users_df\
      .join(movie_rating_df,['user_id'],'inner')\
      .groupBy('name').agg(count('rating').alias('hc'))\
      .orderBy(col('hc').desc(),col('name').asc())\
      .select('name')\
      .limit(1)\
.union(
movies_df\
      .join(movie_rating_df,['movie_id'],'inner')\
      .where('created_at like "2020-02-%"')\
      .groupBy('title').agg(avg('rating').alias('ar'))\
      .orderBy(col('ar').desc(),col('title').asc())\
      .select('title')\
      .limit(1)
).show()




+--------+
|    name|
+--------+
|  Daniel|
|Frozen 2|
+--------+



In [2]:
# In spark SQL

movies_df.createOrReplaceTempView('movies')
movie_rating_df.createOrReplaceTempView('MovieRating')
users_df.createOrReplaceTempView('users')

spark.sql('''
select results
from
(select name as results, count(rating) as cnt from
users join movierating using(user_id)
group by name
order by cnt desc,name asc
limit 1)x

union all

select results
from
(select title as results, avg(rating) as cnt from
movies join movierating using(movie_id)
where created_at like '2020-02%'
group by title
order by cnt desc,title asc
limit 1)y
''').show()

+--------+
| results|
+--------+
|  Daniel|
|Frozen 2|
+--------+



In [21]:
spark.stop()